<a href="https://colab.research.google.com/github/mezba23/2d3d/blob/main/2d3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update"""

In [ ]:
"""!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2"""

In [ ]:
#!apt --fix-broken install

In [ ]:
!nvcc --version

In [ ]:
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
#%load_ext nvcc_plugin

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

11.8
True


In [ ]:
from google.colab import files

filename = list(files.upload().keys())[0]

Saving img3.png to img3.png


In [ ]:
print("/content/"+filename)

In [ ]:
import os

try:
  image_path = "/content/"+filename
except:
  image_path = '/content/drive/MyDrive/5th sem/Mini Project/sample_images/test.png' # example image
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

# output pathes
obj_path = '/content/drive/MyDrive/5th sem/Mini Project/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/drive/MyDrive/5th sem/Mini Project/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = '/content/drive/MyDrive/5th sem/Mini Project/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = '/content/drive/MyDrive/5th sem/Mini Project/results/pifuhd_final/recon/result_%s_256_display.mp4' % file_name

In [ ]:
cd /content/drive/MyDrive/5th sem/Mini Project

/content/drive/MyDrive/5th sem/Mini Project


In [ ]:
#!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

In [ ]:
cd lightweight-human-pose-estimation.pytorch/

/content/drive/MyDrive/5th sem/Mini Project/lightweight-human-pose-estimation.pytorch


In [ ]:
#!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

In [ ]:
#!pip install torch

In [ ]:
#!pip install pycocotools

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

11.8
True


In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
!nvidia-smi

Tue Jun 13 19:36:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)

<ipython-input-10-4884b947f585>:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)


In [ ]:
cd /content/drive/MyDrive/5th sem/Mini Project/

/content/drive/MyDrive/5th sem/Mini Project


In [ ]:
!sh ./scripts/download_trained_model.sh

+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2023-06-13 19:37:48--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.162.119, 52.84.162.20, 52.84.162.103, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.162.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt.4’

pifuhd.pt.4         100%[===================>]   1.44G  72.6MB/s    in 25s     

2023-06-13 19:38:12 (59.8 MB/s) - ‘pifuhd.pt.4’ saved [1548375177/1548375177]

--2023-06-13 19:38:13--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2023-06-13 19:38:13--
Total wall clock time: 25s
Downloaded: 1 files, 1.4G in 25s (59.8 MB/s)


In [ ]:
%set_env PYTHONPATH=/content/drive/MyDrive/5th sem/Mini Project/scripts:/content/drive/MyDrive/5th sem/Mini Project/apps:/content/drive/MyDrive/5th sem/Mini Project:/env/python

env: PYTHONPATH=/content/drive/MyDrive/5th sem/Mini Project/scripts:/content/drive/MyDrive/5th sem/Mini Project/apps:/content/drive/MyDrive/5th sem/Mini Project:/env/python


In [ ]:
# Warning: all images with the corresponding rectangle files under -i will be processed.
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

# seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

Resuming from  ./checkpoints/pifuhd.pt
test data size:  1
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/1 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_img3_256.obj
error cannot marching cubes
cannot unpack non-iterable int object
100% 1/1 [00:06<00:00,  6.55s/it]


In [ ]:
#!pip install pytorch3d

# If you get an error in the next cell, you can instead try the following command (don't forget to comment out the one above!).
# Note that this error is caused by inconsistent cuda version for the pytorch3d package and pytorch in Colab environment.
# Thus, this issue may persist unless pytorch3d in the pip package is updated with the cuda version consistent with pytorch in Colab.
# Also please be aware that the following command is much slower as it builds pytorch3d from scratch.

!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

# You can try another solution below as well. This is also slow and requires you to restart the runtime.

#!pip install 'torch==1.6.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
# !pip install 'torchvision==0.7.0+cu101' -f https://download.pytorch.org/whl/torch_stable.html
#!pip install 'pytorch3d==0.2.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-7b961_ex
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-7b961_ex
  Running command git checkout -q 297020a4b1d7492190cb4a909cafbd2c81a12cb5
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 297020a4b1d7492190cb4a909cafbd2c81a12cb5
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch3d: filename=pytorch3d-0.7.4-cp310-cp310-linux_x86_64.whl size=58477528 sha256=83c358400e71b9b26b9619acabe9f782fa6620ef499809adf94f91de41836c5c
  Stored in dir

In [ ]:
from lib.colab_util import generate_video_from_obj, set_renderer, video

renderer = set_renderer()
generate_video_from_obj(obj_path, out_img_path, video_path, renderer)

# not able to play a mp4 video generated by cv2
!ffmpeg -i $video_path -vcodec libx264 $video_display_path -y -loglevel quiet
video(video_path)

/usr/local/lib/python3.10/dist-packages/pytorch3d/io/obj_io.py:544: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


  0%|          | 0/90 [00:00<?, ?it/s]